In [2]:
import sys
sys.path.append('..')

from helpers.load_data import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [3]:
dfs = load_datafolder("processed")
df = dfs["final_data.csv"]

/home/tanny/vanml-hacks-2023/notebooks/../helpers/load_data.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_filepath, parse_dates=[1, 2])
/home/tanny/vanml-hacks-2023/notebooks/../helpers/load_data.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_filepath, parse_dates=[1, 2])
/home/tanny/vanml-hacks-2023/notebooks/../helpers/load_data.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_filepath, parse_dates=[1, 2])


In [4]:
df.head(n=10)

,callreportnum,DateStart,TimeStart,DateEnd,TimeEnd,CallLength,CityName,CallType,ContactMethod,CallerDemographicsGender,CallerDemographicsAgeGroup,CallerDemographicsCallerType,CallerTypeAffected3rdParty,CallerDemographicsInterpretationLanguage,CallerDemographicsInterpretationOtherlanguage,PublicityOtherPublicity,Mainreason,ReasonDetail
0,54694316.0,2018-07-16,1899-12-31 16:42:00,2018-07-16,1899-12-31 16:51:00,9,New Westminster,Assessment and Referral,Telephone,Female,19-54 Adult,Individual,NaN,English,NaN,Not Obtained,Abuse,Domestic Violence
1,54694316.0,2018-07-16,1899-12-31 16:42:00,2018-07-16,1899-12-31 16:51:00,9,New Westminster,Assessment and Referral,Telephone,Female,19-54 Adult,Individual,NaN,English,NaN,Not Obtained,Housing and Homelessness,Immediate Shelter
2,54693989.0,2018-07-16,1899-12-31 16:27:00,2018-07-16,1899-12-31 16:37:00,10,Winnipeg,Assessment and Referral,Telephone,Female,19-54 Adult,Individual,NaN,English,NaN,Service Provider,Housing and Homelessness,Immediate Shelter
3,54693908.0,2018-07-16,1899-12-31 16:23:00,2018-07-16,1899-12-31 16:26:00,3,Burnaby,Assessment and Referral,Telephone,Female,19-54 Adult,Service Provider,NaN,English,NaN,Not Obtained,Income & Financial Assistance,NaN
4,54693150.0,2018-07-16,1899-12-31 15:54:00,2018-07-16,1899-12-31 15:56:00,2,Vancouver,Assessment and Referral,Telephone,Male,19-54 Adult,Individual,NaN,English,NaN,Repeat Caller,Legal and Public Safety,Legal Advice
5,54693055.0,2018-07-16,1899-12-31 15:50:00,2018-07-16,1899-12-31 15:51:00,1,Vancouver,Assessment and Referral,Telephone,Male,19-54 Adult,Individual,NaN,English,NaN,Repeat Caller,Housing and Homelessness,Immediate Shelter
6,54692972.0,2018-07-16,1899-12-31 15:47:00,2018-07-16,1899-12-31 15:49:00,2,Vancouver,Assessment and Referral,Telephone,Female,19-54 Adult,Individual,NaN,English,NaN,Word of Mouth,Housing and Homelessness,Information on Housing
7,54692972.0,2018-07-16,1899-12-31 15:47:00,2018-07-16,1899-12-31 15:49:00,2,Vancouver,Assessment and Referral,Telephone,Female,19-54 Adult,Individual,NaN,English,NaN,Word of Mouth,Income & Financial Assistance,NaN
8,54692863.0,2018-07-16,1899-12-31 15:42:00,2018-07-16,1899-12-31 15:44:00,2,Vancouver,Info Only/Ref as Req,Telephone,Female,19-54 Adult,Individual,NaN,English,NaN,Not Obtained,Health,NaN
9,54692776.0,2018-07-16,1899-12-31 15:39:00,2018-07-16,1899-12-31 15:47:00,8,Surrey,Assessment and Referral,Telephone,Female,19-54 Adult,Individual,NaN,English,NaN,Phone Book,Health,NaN


In [5]:
features = ['CityName', 'CallerDemographicsGender', 'CallType', 'CallerDemographicsCallerType', 'ContactMethod']
encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), features)
    ],
    remainder='passthrough'  # Keep numeric features that are not transformed
)

X = df[features]
X_transformed = encoder.fit_transform(X)
y = df['CallLength']

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.25, random_state=42)

In [6]:
lr_clf = make_pipeline(
    StandardScaler(with_mean=False),
    LinearRegression()
)

In [7]:
lr_clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('linearregression', LinearRegression())])

In [8]:
lr_clf.score(X_test, y_test)

0.11212679669993997

In [4]:
def train_model_on_city(city_name: str):
    features = ['DateStart','CallerDemographicsGender', 'CallType', 'CallerDemographicsCallerType', 'ContactMethod']
    df = dfs[f'final_data_{city_name}.csv']

    encoder = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown="ignore"), features)
        ],
        remainder='passthrough'  # Keep numeric features that are not transformed
    )
    X = df[features] 
    y = df["CallLength"]
    X_transform = encoder.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_transform, y, test_size=0.2, random_state=42)
    
    rf_pipeline = make_pipeline(
        StandardScaler(with_mean=False),
        LinearRegression()
    )   

    return rf_pipeline.fit(X_train, y_train), X_test, y_test

In [5]:
cities = ['burnaby', 'nw', 'surrey', 'vancouver', 'victoria']

for city in cities:
    rf_model, X_test, y_test = train_model_on_city(city)
    print(f'{city} has a score of {rf_model.score(X_test, y_test)}')

burnaby has a score of 0.18956287111261516
nw has a score of 0.15950388994931863
surrey has a score of 0.15005020241749667
vancouver has a score of 0.13230706074636944
victoria has a score of 0.0650497577285647
